In [73]:
# Simulate the Thomas attractor
# https://en.wikipedia.org/wiki/Thomas%27_cyclically_symmetric_attractor  
b::Float64 = 0.15

function thomas_rhs(x::AbstractVector{Float64})::Vector{Float64}
    return sin.(circshift(x, -1)) .- b .* x
end

dt::Float64 = 0.05  # timetep
n::Int = 1024 * 32  # steps

# Do RK4 to fill out a trajectory
data::Matrix{Float64} = zeros(Float64, n, 3)
data[1, :] = [1.0, 0.0, 0.0]  # initial data

for i::Int = 2:n
    x_prev::Vector{Float64} = data[i-1, :]
    k1::Vector{Float64} = dt .* thomas_rhs(x_prev)
    k2::Vector{Float64} = dt .* thomas_rhs(x_prev .+ k1 ./ 2)
    k3::Vector{Float64} = dt .* thomas_rhs(x_prev .+ k2 ./ 2)
    k4::Vector{Float64} = dt .* thomas_rhs(x_prev .+ k3)
    data[i, :] = x_prev .+ (k1 .+ 2 .* k2 .+ 2 .* k3 .+ k4) ./ 6
end

# Name each coordinate
x::Vector{Float64} = data[:, 1]
y::Vector{Float64} = data[:, 2]
z::Vector{Float64} = data[:, 3]

x,y,z

([1.0, 0.9925454644096016, 0.9852502329841146, 0.9782146007704968, 0.9715357486062538, 0.9653076373802497, 0.9596209070538471, 0.9545627759457646, 0.9502169366277917, 0.9466634455771092  …  -0.8805543490352534, -0.8265402284673823, -0.7728869834177109, -0.7195654720341539, -0.6665495260139652, -0.6138163253060125, -0.5613466573324399, -0.5091250648001334, -0.45713988836958364, -0.40538321221688967], [0.0, 0.0010447578983220384, 0.0041497163938394865, 0.009268848163508469, 0.016353730831790307, 0.025353957622295503, 0.03621752717547331, 0.04889120839331185, 0.06332087662189681, 0.07945181796744766  …  1.8818379289253995, 1.8799513175576035, 1.876211925491809, 1.8707238143377027, 1.8635983653975066, 1.8549530495809186, 1.8449102689402213, 1.8335962825588372, 1.8211402251824347, 1.8076732229935017], [0.0, 0.041815094294385007, 0.08311674499780629, 0.1239123289373594, 0.1642121261620346, 0.20402930897728735, 0.24337990872123474, 0.28228275796624713, 0.32075940625333926, 0.3588340078637479 

In [ ]:
include("../../functions/GreedyRegression.jl")
include("../../functions/Integrate.jl")
include("../../functions/ReportModel.jl")
include("../../functions/EnvelopePol.jl")
include("../../functions/PickSubdomains.jl")

using .GreedyRegression, .Integrate, .ReportModel, .EnvelopePol, .PickSubdomains

nl::Int = 2 # under-estimate this
nw::Int = 128 # number of domains we integrate over 
dof::Int = 1 # scalars have one degree of freedom
dim::Int = 1 # how many dimensions does our data have?
env_pow::Int = 4 # weight is (1-x^2)^power
size_vec::Vector{Int} = [128] # how many gridpoints should we use per integration?
buffer::Int = 0 # Don't use points this close to boundary

pol::Matrix{Float64} = EnvelopePol.envelope_pol(env_pow, dim)

size_of_data::Vector{Int} = [size(data,1)]
seed::Int = 100
corners::Matrix{Int} = PickSubdomains.pick_subdomains(size_of_data, size_vec, buffer, nw, seed)

t::StepRangeLen = dt*(0:n-1)
grid::Vector{Vector{Float64}} = [Vector{Float64}(t)]
size(corners)

(1, 128)

In [75]:
using Statistics

G::Matrix{Float64} = zeros(Float64, dof*nw, 0)
labels::Vector{String} = String[]
scales::Vector{Float64} = Float64[]

function add_library_term(label::AbstractString, data::AbstractArray, derivs::AbstractVector{Int64}, scale::AbstractFloat)
    col::Vector{Float64} = Integrate.integrate(data, derivs, grid, corners, size_vec, pol)

    global G = hcat(G, col)
    push!(labels, label)
    push!(scales, scale)
end

add_library_term("dx/dt", x, [1], mean(abs.(x)))
add_library_term("dy/dt", y, [1], mean(abs.(y)))
add_library_term("dz/dt", z, [1], mean(abs.(z)))

add_library_term("x", x, Int64[], 1.)
add_library_term("y", y, Int64[], 1.)
add_library_term("z", z, Int64[], 1.)

add_library_term("sin(x)", sin.(x), Int64[], 1.)
add_library_term("sin(y)", sin.(y), Int64[], 1.)
add_library_term("sin(z)", sin.(z), Int64[], 1.)

norm_vec::Vector{Float64} = Integrate.integrate(0*x .+ 1, Int64[], grid, corners, size_vec, pol)
G ./= norm_vec
G ./= scales'

G

128×9 Matrix{Float64}:
 -0.198413    0.550385     …   0.814099   -0.121245    0.686568
  0.0352966  -0.0453813       -0.872077   -0.238973   -0.525147
  0.291411   -0.527564        -0.561307    0.461122   -0.917116
 -0.489496    0.000542453     -0.0230263  -0.982016   -0.251538
  0.548228    0.151879        -0.492686    0.92705     0.561389
  0.0899302   0.200551     …   0.886343    0.480762    0.494956
  0.101894    0.180735        -0.651825    0.785477    0.680286
 -0.321385   -0.0181539        0.286978   -0.589235    0.529717
  0.454188    0.272394        -0.603276    0.742769    0.707359
  0.320528    0.357664         0.563354    0.762664    0.924859
  0.0388023   0.171863     …  -0.593682    0.645873    0.701241
 -0.125976    0.297543        -0.927862   -0.524254    0.238094
  0.061986    0.0945152       -0.726783    0.719339    0.530901
  ⋮                        ⋱                          
 -0.199145    0.394262        -0.873841   -0.678038    0.510547
 -0.333236   -0.258577    

In [76]:
using LinearAlgebra

gamma::Float64 = 1.25

for i::Int = 1:3
    cs::Matrix{Float64}, residuals::Vector{Float64}, _ = GreedyRegression.greedy_regression(G)
    k::Int = ReportModel.report_identified_model(cs, residuals, scales, labels, gamma)

    col_norms::Vector{Float64} = [norm(G[:, j] .* cs[j, k]) for j = 1:size(G, 2)]
    kill::Int = argmax(col_norms)
    G = G[:, setdiff(1:end, kill)]
    labels = labels[setdiff(1:end, kill)]
    scales = scales[setdiff(1:end, kill)]
end

[-0.9999999503147231, -0.15000000107885458, 1.0]
["dx/dt", "x", "sin(y)"]


dx/dt + 0.15x - sin(y) = 0


[-0.9999999490645282, -0.1500000009111534, 1.0]
["dy/dt", "y", "sin(z)"]


dy/dt + 0.15y - sin(z) = 0


[0.9999999480090354, 0.15000000172283903, -1.0]
["dz/dt", "z", "sin(x)"]


dz/dt + 0.15z - sin(x) = 0


